In [17]:
import numpy as np
import pandas as pd
import os
from pprint import pprint

def load_player_info(path):
    return pd.read_csv(path + "/info.csv")

def load_player_seasons(path):
    return pd.read_csv(path + "/seasons.csv")

def get_team_switches(path):
    idf = load_player_info(path)
    if not "Forward" in idf["primaryPosition"][0]:
        return
    sdf = load_player_seasons(path)
    sdf = sdf[sdf["league"] == "National Hockey League"]
    sdf["team_switch"] = (sdf["team"].shift(1, fill_value = 
                          sdf["team"].head(1)) != sdf["team"])
    
    if not sdf.empty:
        sdf = sdf.reset_index(drop=True)
        switch_idx = np.r_[sdf[sdf["team_switch"] == True].index]
        if len(switch_idx) > 0:
            selected_cols = np.array([switch_idx - 1, switch_idx]).flatten('F')
            
            sdf = sdf.iloc[selected_cols]
            sdf = sdf.reset_index(drop=True)
            if idf.fullName[0] == "Blake Coleman":
                print(sdf["team_switch"])
                print(sdf["team"].shift(1) != sdf["team"])
                print((sdf["team_switch"]) & (sdf["team"].shift(1) != sdf["team"]))
            sdf["team_switch"] = (sdf["team_switch"]) & (sdf["team"].shift(1) != sdf["team"])
            sdf["fullName"] = np.resize(idf.fullName,len(sdf))
            sdf["diff"] = sdf["goals"] - sdf["goals"].shift(1)
            
            return sdf
        else:
            return
    else:
        return


if __name__ == "__main__":
    data_dir = "/home/fileds/data/NHL/player_data/data/"
    player_dirs = os.listdir(data_dir)
    switch_dfs = []
    for d in player_dirs:
        df = get_team_switches(data_dir + d)
        if df is not None:
            switch_dfs.append(df)
            
    dfs = pd.concat([df for df in switch_dfs])
    dfs = dfs.reset_index(drop=True)
    



0    False
1     True
2    False
3     True
Name: team_switch, dtype: bool
0     True
1     True
2    False
3     True
Name: team, dtype: bool
0    False
1     True
2    False
3     True
dtype: bool


In [18]:
np.mean(dfs[dfs["team_switch"] == True]["diff"].dropna())

0.847870182555781

In [22]:
print(np.sum(dfs.team_switch))
print(len(dfs.index))
print(len(dfs.index) / np.sum(dfs.team_switch))

493
986
2.0


In [34]:
dfs.diff = np.where(dfs.team_switch == False, np.nan)

ValueError: either both or neither of x and y should be given

In [26]:
pd.options.display.max_rows = 999
dfs[["fullName", "season", "team", "diff", "goals", "team_switch"]]

,fullName,season,team,diff,goals,team_switch
0,Conor Sheary,20172018,Pittsburgh Penguins,NaN,18,False
1,Conor Sheary,20182019,Buffalo Sabres,-4.0,14,True
2,Conor Sheary,20192020,Buffalo Sabres,-5.0,9,False
3,Conor Sheary,20192020,Pittsburgh Penguins,-8.0,1,True
4,Conor Sheary,20192020,Pittsburgh Penguins,0.0,1,False
5,Conor Sheary,20202021,Washington Capitals,13.0,14,True
6,Taylor Raddysh,20212022,Tampa Bay Lightning,NaN,5,False
7,Taylor Raddysh,20212022,Chicago Blackhawks,1.0,6,True
8,Jason Dickinson,20202021,Dallas Stars,NaN,7,False
9,Jason Dickinson,20212022,Vancouver Canucks,-2.0,5,True


In [191]:
dfs.to_csv("team_switch.csv")